# Grand Challenge DRIVE challenge: Digital Retinal Images for Vessel Extraction

The DRIVE database has been established to enable comparative studies on segmentation of blood vessels in retinal images. Retinal vessel segmentation and delineation of morphological attributes of retinal blood vessels, such as length, width, tortuosity, branching patterns and angles are utilized for the diagnosis, screening, treatment, and evaluation of various cardiovascular and ophthalmologic diseases such as diabetes, hypertension, arteriosclerosis and chorodial neovascularization. Automatic detection and analysis of the vasculature can assist in the implementation of screening programs for diabetic retinopathy, can aid research on the relationship between vessel tortuosity and hypertensive retinopathy, vessel diameter measurement in relation with diagnosis of hypertension, and computer-assisted laser surgery. Automatic generation of retinal maps and extraction of branch points have been used for temporal or multimodal image registration and retinal image mosaic synthesis. Moreover, the retinal vascular tree is found to be unique for each individual and can be used for biometric identification.
Data

The photographs for the DRIVE database were obtained from a diabetic retinopathy screening program in The Netherlands. The screening population consisted of 400 diabetic subjects between 25-90 years of age. Forty photographs have been randomly selected, 33 do not show any sign of diabetic retinopathy and 7 show signs of mild early diabetic retinopathy. 

## Setup

This notebook was created and tested on an ml.t3.medium notebook instance.

Let's start by creating a SageMaker session and specifying:

* The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
* The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the sagemaker.get_execution_role() with a the appropriate full IAM role arn string(s).

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'radboud-immersion-day'

role = sagemaker.get_execution_role()

## Data

Since the data is stored in ZIP files, we need the `unzip` tool to unpack the datasets. We will install it using the `apt-get` command by first fetching repository data, and then issuing the actual install command.

In [ ]:
!apt-get update && apt-get install -y unzip

Let's fetch the data now, and unpack it into the `data` directory.

In [ ]:
# Delete any previous directory containing the dataset.
!rm -Rf data
# Create the data directory.
!mkdir data
# Fetch both the training and test datasets from their locations.
!curl -L -o ./data/training.zip https://www.dropbox.com/sh/z4hbbzqai0ilqht/AADp_8oefNFs2bjC2kzl2_Fqa/training.zip?dl=0
!curl -L -o ./data/test.zip https://www.dropbox.com/sh/z4hbbzqai0ilqht/AABuUJQJ5yG5oCuziYzYu8jWa/test.zip?dl=0
# Unzip the data within the data directory and delete the zip files.
!cd data && unzip training.zip && unzip test.zip && rm *.zip && cd ..

### Copy the data to S3

We are going to use the sagemaker.Session.upload_data function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.

In [ ]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

## Train

### Training script

The radboud.py script provides all the code we need for training and hosting a SageMaker model (model_fn function to load a model). The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

    SM_MODEL_DIR: A string representing the path to the directory to write model artifacts to. These artifacts are uploaded to S3 for model hosting.
    SM_NUM_GPUS: The number of gpus available in the current container.
    SM_CURRENT_HOST: The name of the current container on the container network.
    SM_HOSTS: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's fit() method, the following will be set, following the format SM_CHANNEL_\[channel_name\]:

    SM_CHANNEL_TRAINING: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit SageMaker Containers.

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to model_dir so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an argparse.ArgumentParser instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (if __name__=='__main__':) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

In [ ]:
!pygmentize radboud.py

### Run training in SageMaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters. In this case we are going to run our training job on an `ml.p3.2xlarge` instance. But this example can be run on other CPU or GPU instance types as well. The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the mnist.py script above.

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='radboud.py',
                    role = role,
                    framework_version='1.6.0',
                    py_version='py3',
                    train_instance_count=1,
                    train_instance_type='ml.p3.2xlarge',
                    hyperparameters={
                        'epochs': 15,
                        'batch-size': 3,
                    })

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.

In [ ]:
estimator.fit({'training': inputs})

## Host

### Create endpoint

After training, we use the PyTorch estimator object to build and deploy a PyTorchPredictor. This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

As mentioned above we have implementation of model_fn in the mnist.py script that is required. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fn` defined in sagemaker-pytorch-containers.

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances, but you need to make sure that you return or save your model as a cpu model similar to what we did in `radboud.py`. Here we will deploy the model to a single `ml.c5.xlarge` instance.

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')

### Evaluate

We can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a data variable in this notebook, which we can then pass to the predictor.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import torchvision.transforms as transforms
%matplotlib inline


test_img = Image.open('./data/training/images/21_training.tif')
test_mask = Image.open('./data/training/1st_manual/21_manual1.gif')
test_mask = test_mask.convert('L')

transform = transforms.Compose([
    transforms.Resize((258, 250), 2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_input = transform(test_img)
test_input = test_input.unsqueeze(0)

prediction = predictor.predict(test_input)
prediction = np.atleast_1d(prediction)

fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(16, 6))

ax0.imshow(transforms.Resize((216, 244), 2)(test_img))
#ax0.imshow(np.array(test_img) / 255.0)
ax1.imshow(np.array(test_mask))
ax2.imshow(prediction[0]['out'].squeeze().numpy(), cmap='jet')

plt.show()

### Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it

In [ ]:
predictor.delete_endpoint()